# Annotate multiple plots

Let's start by importing the libraries we'll need for the plot and create the plot. We will here create a plots, where speed have been cutoff if it under 100.

In [ ]:
import hvplot.pandas  # noqa: F401
import numpy as np
import pandas as pd

raw_speed_data = pd.read_parquet("../assets/example.parquet")
raw_speed_curve = raw_speed_data.hvplot("TIME", "SPEED")

filt_speed_data = raw_speed_data.copy()
filt_speed_data[raw_speed_data.SPEED < 100] = pd.NA
filt_speed_curve = filt_speed_data.hvplot("TIME", "SPEED")

raw_speed_curve + filt_speed_curve


## Shared key dimension

Here we will use the fact that the key dimension of both plot is `TIME`, and create the annotator based on one of the plots. We will then overlaid that annotaotr on top of both plots. 

If you already have run the [previous example](00_first_annotation) you will see some familiar annotations.

In [ ]:
from holonote.annotate import Annotator

annotator = Annotator(filt_speed_curve, fields=['description'])
annotator * raw_speed_curve + annotator * filt_speed_curve


When you highlight one of the annotations, it will show up on both plots. Lets highlight the region around the 19th of July on the filtered curve.

In [ ]:
annotator.add_annotation(description="Missing data on filtered data")


To save the data to a database and persist the data we use the `.commit` method.

In [ ]:
annotator.commit()


## Different key dimensions

Another way to annotate multiple plots is where the plots have different key dimensions. An example of this could be if we have a [power spectral density](https://en.wikipedia.org/wiki/Spectral_density) (PSD) of the data. The PSD is a function of frequency, and we would like to annotate the PSD with the same annotations as the time series.

Lets create the PSD first, here we will use SciPy's [welch](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html) function to calculate the PSD. The sampling frequency (fs) is 1 / (60 seconds). We will use HoloViews to plot the PSD in a log-log plot.

In [ ]:
import holoviews as hv
from scipy.signal import welch

f, Pxx = welch(raw_speed_data.SPEED.fillna(0), fs=1 / 60, nperseg=1024, noverlap=20)
welch_curve = hv.Curve((f[1:], Pxx[1:]), "Frequency", "Power").opts(logx=True, logy=True, tools=["hover"])
welch_curve


We will now create a new annotator which accounts for the fact that there is a new key dimension, frequency. This is done by passing in the specification directly.

In [ ]:
from holonote.annotate import Annotator

specification = {"TIME": np.datetime64, "Frequency": float}
multi_annotator = Annotator(specification, fields=['description'])


You should now be able to annotate both on the time series plot and the PSD plot.  

In [ ]:
multi_annotator * filt_speed_curve + multi_annotator * welch_curve


And then again we can add a description to the annotation and save the data to the database.


In [ ]:
multi_annotator.add_annotation(description="Missing data in time and spike in filtered data")
multi_annotator.commit()
